<a href="https://colab.research.google.com/github/Nishilhmodi/Education-Recommendation-System/blob/main/Education_Recommendation_sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
---
#**Education Recommendation System**
---
---



**Importing the required libraries**



In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

**Loading dataset**

In [3]:
dataset1 = pd.read_csv('/content/Student-scores.csv')
dataset = dataset1.copy()
dataset.head()

,id,first_name,last_name,email,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score
0,1,Paul,Casey,paul.casey.1@gslingacademy.com,male,False,3,False,27,Lawyer,73,81,93,97,63,80,87
1,2,Danielle,Sandoval,danielle.sandoval.2@gslingacademy.com,female,False,2,False,47,Doctor,90,86,96,100,90,88,90
2,3,Tina,Andrews,tina.andrews.3@gslingacademy.com,female,False,9,True,13,Government Officer,81,97,95,96,65,77,94
3,4,Tara,Clark,tara.clark.4@gslingacademy.com,female,False,5,False,3,Artist,71,74,88,80,89,63,86
4,5,Anthony,Campos,anthony.campos.5@gslingacademy.com,male,False,5,False,10,Unknown,84,77,65,65,80,74,76


**Dropping the irrelevant columns**

In [4]:
dataset.drop(columns=['id','first_name','last_name','email'], axis=1, inplace=True)

In [5]:
dataset.columns

Index(['gender', 'part_time_job', 'absence_days', 'extracurricular_activities',
       'weekly_self_study_hours', 'career_aspiration', 'math_score',
       'history_score', 'physics_score', 'chemistry_score', 'biology_score',
       'english_score', 'geography_score'],
      dtype='object')

**Creating a new feature from all score**

In [6]:
dataset['Total score'] = dataset['math_score'] + dataset['history_score'] + dataset['physics_score'] + dataset['chemistry_score'] + dataset['biology_score'] + dataset['english_score'] + dataset['geography_score']
dataset['Average score'] = dataset['Total score']/7
dataset.head()

,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score,Total score,Average score
0,male,False,3,False,27,Lawyer,73,81,93,97,63,80,87,574,82.000000
1,female,False,2,False,47,Doctor,90,86,96,100,90,88,90,640,91.428571
2,female,False,9,True,13,Government Officer,81,97,95,96,65,77,94,605,86.428571
3,female,False,5,False,3,Artist,71,74,88,80,89,63,86,551,78.714286
4,male,False,5,False,10,Unknown,84,77,65,65,80,74,76,521,74.428571


In [7]:
dataset['career_aspiration'].unique()
dataset['career_aspiration'].value_counts()

,count
career_aspiration,
Software Engineer,315
Business Owner,309
Unknown,223
Banker,169
Lawyer,138
Accountant,126
Doctor,119
Real Estate Developer,83
Stock Investor,73


In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   gender                      2000 non-null   object 
 1   part_time_job               2000 non-null   bool   
 2   absence_days                2000 non-null   int64  
 3   extracurricular_activities  2000 non-null   bool   
 4   weekly_self_study_hours     2000 non-null   int64  
 5   career_aspiration           2000 non-null   object 
 6   math_score                  2000 non-null   int64  
 7   history_score               2000 non-null   int64  
 8   physics_score               2000 non-null   int64  
 9   chemistry_score             2000 non-null   int64  
 10  biology_score               2000 non-null   int64  
 11  english_score               2000 non-null   int64  
 12  geography_score             2000 non-null   int64  
 13  Total score                 2000 

In [9]:
dataset['gender'] = dataset['gender'].replace(['male', 'female'], [1, 0])
dataset['part_time_job'] = dataset['part_time_job'].replace([True, False], [1, 0])
dataset['extracurricular_activities'] = dataset['extracurricular_activities'].replace([True, False], [1, 0])
dataset['career_aspiration'] = dataset['career_aspiration'].replace(['Lawyer', 'Doctor', 'Government Officer', 'Artist', 'Unknown',
       'Software Engineer', 'Teacher', 'Business Owner', 'Scientist',
       'Banker', 'Writer', 'Accountant', 'Designer',
       'Construction Engineer', 'Game Developer', 'Stock Investor',
       'Real Estate Developer'], [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])
dataset.head()

/tmp/ipython-input-9-3683102732.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset['gender'] = dataset['gender'].replace(['male', 'female'], [1, 0])
/tmp/ipython-input-9-3683102732.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset['part_time_job'] = dataset['part_time_job'].replace([True, False], [1, 0])
/tmp/ipython-input-9-3683102732.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(cop

,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score,Total score,Average score
0,1,0,3,0,27,0,73,81,93,97,63,80,87,574,82.000000
1,0,0,2,0,47,1,90,86,96,100,90,88,90,640,91.428571
2,0,0,9,1,13,2,81,97,95,96,65,77,94,605,86.428571
3,0,0,5,0,3,3,71,74,88,80,89,63,86,551,78.714286
4,1,0,5,0,10,4,84,77,65,65,80,74,76,521,74.428571


In [10]:
dataset.shape

(2000, 15)

**Balancing the dataset**

In [11]:
smote = SMOTE(random_state=42)
X = dataset.drop('career_aspiration', axis=1)
y = dataset['career_aspiration']
X_res, y_res = smote.fit_resample(X, y)

In [12]:
y_res.value_counts()

,count
career_aspiration,
0,315
1,315
2,315
3,315
4,315
5,315
6,315
7,315
8,315


**Train - Test split**

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Feature scalling**

In [14]:
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)
x_train

array([[-0.99252792, -0.43440451, -0.6382063 , ..., -1.80405955,
         0.55004798,  0.55004798],
       [ 1.00752834, -0.43440451, -1.39656506, ...,  0.53599067,
         1.25192148,  1.25192148],
       [ 1.00752834,  2.30200187, -0.6382063 , ..., -1.63072249,
        -2.98352206, -2.98352206],
       ...,
       [ 1.00752834, -0.43440451,  0.12015247, ..., -0.24402607,
        -0.70848382, -0.70848382],
       [-0.99252792,  2.30200187,  0.49933185, ..., -0.07068902,
        -0.99891423, -0.99891423],
       [ 1.00752834, -0.43440451,  2.39522875, ..., -1.02404281,
        -1.28934465, -1.28934465]])

In [15]:
model = {
'LogisticRegression': LogisticRegression(),
'SVC': SVC(),
'RandomForestClassifier': RandomForestClassifier(),
'KNeighborsClassifier': KNeighborsClassifier(),
'DecisionTreeClassifier': DecisionTreeClassifier(),
'GaussianNB': GaussianNB(),
'AdaBoostClassifier': AdaBoostClassifier(),
'GradientBoostingClassifier': GradientBoostingClassifier()
}

for name, model in model.items():
    print("="*50)
    print("Model_name : ",name)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("Accuracy_score : ",accuracy_score(y_test, y_pred))
    print("Classification_report : \n",classification_report(y_test, y_pred))
    print("Confusion_matrix : \n",confusion_matrix(y_test, y_pred))

Model_name :  LogisticRegression
Accuracy_score :  0.4475
Classification_report : 
               precision    recall  f1-score   support

           0       0.43      0.43      0.43        28
           1       0.47      0.72      0.57        25
           2       0.00      0.00      0.00        13
           3       0.43      0.21      0.29        14
           4       0.33      0.35      0.34        52
           5       0.34      0.55      0.42        53
           6       0.25      0.12      0.17        16
           7       0.86      0.94      0.90        66
           8       0.33      0.10      0.15        10
           9       0.23      0.17      0.19        30
          10       0.25      0.20      0.22         5
          11       0.69      0.46      0.55        24
          12       0.00      0.00      0.00        16
          13       0.31      0.42      0.36        12
          14       0.43      0.75      0.55        12
          15       0.00      0.00      0.00        

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy_score :  0.4325
Classification_report : 
               precision    recall  f1-score   support

           0       0.45      0.50      0.47        28
           1       0.48      0.84      0.61        25
           2       0.00      0.00      0.00        13
           3       0.33      0.21      0.26        14
           4       0.26      0.23      0.24        52
           5       0.33      0.70      0.45        53
           6       0.00      0.00      0.00        16
           7       0.85      0.92      0.88        66
           8       0.00      0.00      0.00        10
           9       0.14      0.13      0.14        30
          10       0.00      0.00      0.00         5
          11       0.82      0.38      0.51        24
          12       0.00      0.00      0.00        16
          13       1.00      0.08      0.15        12
          14       0.53      0.67      0.59        12
          15       0.00      0.00      0.00        10
          16       0.15      0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy_score :  0.47
Classification_report : 
               precision    recall  f1-score   support

           0       0.44      0.61      0.51        28
           1       0.56      0.76      0.64        25
           2       0.20      0.08      0.11        13
           3       0.33      0.36      0.34        14
           4       0.31      0.29      0.30        52
           5       0.41      0.74      0.52        53
           6       0.33      0.12      0.18        16
           7       0.88      0.95      0.91        66
           8       0.00      0.00      0.00        10
           9       0.13      0.13      0.13        30
          10       0.00      0.00      0.00         5
          11       0.75      0.50      0.60        24
          12       0.00      0.00      0.00        16
          13       0.00      0.00      0.00        12
          14       0.47      0.67      0.55        12
          15       0.00      0.00      0.00        10
          16       0.23      0.2

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

Accuracy_score :  0.325
Classification_report : 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        28
           1       0.00      0.00      0.00        25
           2       0.00      0.00      0.00        13
           3       0.00      0.00      0.00        14
           4       0.20      0.46      0.27        52
           5       0.21      0.60      0.31        53
           6       0.00      0.00      0.00        16
           7       0.58      1.00      0.73        66
           8       0.00      0.00      0.00        10
           9       0.00      0.00      0.00        30
          10       0.00      0.00      0.00         5
          11       1.00      0.33      0.50        24
          12       0.00      0.00      0.00        16
          13       0.00      0.00      0.00        12
          14       0.00      0.00      0.00        12
          15       0.00      0.00      0.00        10
          16       0.00      0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy_score :  0.5025
Classification_report : 
               precision    recall  f1-score   support

           0       0.53      0.68      0.59        28
           1       0.70      0.84      0.76        25
           2       0.11      0.08      0.09        13
           3       0.27      0.29      0.28        14
           4       0.38      0.29      0.33        52
           5       0.45      0.77      0.57        53
           6       0.44      0.25      0.32        16
           7       0.91      0.91      0.91        66
           8       1.00      0.30      0.46        10
           9       0.25      0.30      0.27        30
          10       0.00      0.00      0.00         5
          11       0.79      0.46      0.58        24
          12       0.12      0.06      0.08        16
          13       0.12      0.08      0.10        12
          14       0.42      0.67      0.52        12
          15       0.00      0.00      0.00        10
          16       0.21      0

**Selecting the best model for the dataset**

In [16]:
model = RandomForestClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print("Accuracy_score : ",accuracy_score(y_test, y_pred))
print("Classification_report : \n",classification_report(y_test, y_pred))
print("Confusion_matrix : \n",confusion_matrix(y_test, y_pred))

Accuracy_score :  0.4875
Classification_report : 
               precision    recall  f1-score   support

           0       0.42      0.50      0.46        28
           1       0.57      0.80      0.67        25
           2       0.00      0.00      0.00        13
           3       0.42      0.36      0.38        14
           4       0.30      0.27      0.29        52
           5       0.43      0.87      0.58        53
           6       0.20      0.12      0.15        16
           7       0.86      0.97      0.91        66
           8       0.00      0.00      0.00        10
           9       0.21      0.20      0.21        30
          10       0.00      0.00      0.00         5
          11       0.76      0.54      0.63        24
          12       0.00      0.00      0.00        16
          13       0.00      0.00      0.00        12
          14       0.47      0.67      0.55        12
          15       0.00      0.00      0.00        10
          16       0.27      0

In [17]:
print("Predicted label :",model.predict(x_test[40].reshape(1,-1)))
print("Actual :",y_test.iloc[40])

Predicted label : [4]
Actual : 4


**Saving and Loading the files**

In [18]:
import pickle

# SAVE FILES
scaler = StandardScaler()
pickle.dump(scaler,open("scaler.pkl",'wb'))
pickle.dump(model,open("model.pkl",'wb'))

In [19]:
# Load the scaler, label encoder, and model
scaler = pickle.load(open("scaler.pkl", 'rb'))
model = pickle.load(open("model.pkl", 'rb'))

---

#**Recommendation System**

---

In [20]:
import pickle
import numpy as np

# Load the scaler, label encoder, model, and class names
scaler = pickle.load(open("scaler.pkl", 'rb'))
model = pickle.load(open("model.pkl", 'rb'))

class_names = ['Lawyer', 'Doctor', 'Government Officer', 'Artist', 'Unknown',
               'Software Engineer', 'Teacher', 'Business Owner', 'Scientist',
               'Banker', 'Writer', 'Accountant', 'Designer',
               'Construction Engineer', 'Game Developer', 'Stock Investor',
               'Real Estate Developer']

def Recommendations(gender, part_time_job, absence_days, extracurricular_activities,
                    weekly_self_study_hours, math_score, history_score, physics_score,
                    chemistry_score, biology_score, english_score, geography_score,
                    total_score,average_score):

    # Encode categorical variables
    gender_encoded = 1 if gender.lower() == 'male' else 0
    part_time_job_encoded = 1 if part_time_job else 0
    extracurricular_activities_encoded = 1 if extracurricular_activities else 0

    # Create feature array
    feature_array = np.array([[gender_encoded, part_time_job_encoded, absence_days, extracurricular_activities_encoded,
                               weekly_self_study_hours, math_score, history_score, physics_score,
                               chemistry_score, biology_score, english_score, geography_score,total_score,average_score]])

    if not hasattr(scaler, 'mean_'):
        print("Warning: Scaler is not fitted. You need to fit it to some data before using it.")
    sample_data = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
    scaler.fit(sample_data)

    # Scale features
    scaled_features = scaler.transform(feature_array)

    # Predict using the model
    probabilities = model.predict_proba(scaled_features)

    # Get top five predicted classes along with their probabilities
    top_classes_idx = np.argsort(-probabilities[0])[:5]
    top_classes_names_probs = [(class_names[idx], probabilities[0][idx]) for idx in top_classes_idx]

    return top_classes_names_probs

In [28]:
final_recommendations = Recommendations(gender='male',
                                        part_time_job=False,
                                        absence_days=2,
                                        extracurricular_activities=False,
                                        weekly_self_study_hours=1,
                                        math_score=79,
                                        history_score=33,
                                        physics_score=38,
                                        chemistry_score=39,
                                        biology_score=40,
                                        english_score=33,
                                        geography_score=56,
                                        total_score=(79 + 33 + 38 + 39 + 40 + 33 + 56),
                                        average_score=(79 + 33 + 38 + 39 + 40 + 33 + 56)/7)

print("Top recommended studies with probabilities:")
print("="*50)
for class_name, probability in final_recommendations:
    print(f"{class_name} with probability {probability * 100}")

Top recommended studies with probabilities:
Software Engineer with probability 30.0
Doctor with probability 21.0
Unknown with probability 13.0
Lawyer with probability 8.0
Banker with probability 8.0


In [29]:
final_recommendations = Recommendations(gender='male',
                                        part_time_job=True,
                                        absence_days=2,
                                        extracurricular_activities=True,
                                        weekly_self_study_hours=10,
                                        math_score=79,
                                        history_score=33,
                                        physics_score=38,
                                        chemistry_score=39,
                                        biology_score=40,
                                        english_score=33,
                                        geography_score=56,
                                        total_score=(79 + 33 + 38 + 39 + 40 + 33 + 56),
                                        average_score=(79 + 33 + 38 + 39 + 40 + 33 + 56)/7)

print("Top recommended studies with probabilities:")
print("="*50)
for class_name, probability in final_recommendations:
    print(f"{class_name} with probability {probability * 100}")

Top recommended studies with probabilities:
Doctor with probability 52.0
Software Engineer with probability 17.0
Unknown with probability 9.0
Banker with probability 7.000000000000001
Accountant with probability 4.0


In [ ]:
import sklearn
sklearn.__version__

'1.3.2'